# Fashion Mnist DNN Tutorial [CNN & Multi-layer Perceptron (MLP)]

Efficient Network
- https://github.com/lukemelas/EfficientNet-PyTorch
- image 시리즈 중 auto ML을 통해 효율적으로 구현한 것

pytorchLightning & Hydra
- config_utils.py 파일 사용할 예정 : 해당 파일 import 하기
- 아래 코드에서 중복되는 부분은 모두 삭제 : class WarmupLR, class EarlyStopping, def softmax
- https://pytorch-lightning.readthedocs.io/en/latest/common/lightning_module.html

외부 파일 가져오기 & requirements 설치

In [ ]:
!pwd

# mount
from google.colab import drive
drive.mount('/content/drive')

import os
import sys

drive_project_root = '/content/drive/MyDrive/#fastcampus'
sys.path.append(drive_project_root)

!ls

!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

In [ ]:
!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

In [ ]:
pip install torch-optimizer

wandb 오류 있을 때 : `wandb.flush`

In [ ]:
pip install wandb

In [ ]:
pip install omegaconf

In [ ]:
pip install efficientnet_pytorch

https://pytorch.org/vision/stable/transforms.html

★ OmegaConf
- https://omegaconf.readthedocs.io/en/2.1_branch/
- hyperparameter configuration을 관리하기 위한 open source library
- DictConfig : Dictionary 형태의 configuration
- Hydra도 omegaconf를 기반으로 만들어짐
  - Hydra는 무겁기 때문에 omegaconf를 먼저 거치고, hydra 사용

gpu 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

In [ ]:
pip install Hydra

In [ ]:
pip install hydra-core

In [ ]:
pip uninstall torchmetrics

In [ ]:
pip install torchmetrics==0.5

In [ ]:
pip install pytorch-lightning

In [ ]:
# 추상화를 하는 class -> 중복 코드를 없애기 위함
from abc import abstractmethod, ABC

from typing import Optional, Dict, List, Union, Any, Optional, Iterable, Callable
from functools import partial
from collections import Counter, OrderedDict
from datetime import datetime
import random
import math

import numpy as np
from tqdm import tqdm   # 진행율
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F     # relu 등 함수 모음
from torch.utils.data import DataLoader
import pytorch_lightning as pl

# data & models
from torchvision.datasets import FashionMNIST
from torchvision import transforms    # feature engineering 전처리 작업 효율적으로 할 수 있게 도와줌
import torchvision.utils as vutils   # for 이미지 저장

# For configuration
from omegaconf import OmegaConf, DictConfig
import hydra
from hydra.core.config_store import ConfigStore

# For logger
from torch.utils.tensorboard import SummaryWriter
import wandb
os.environ["WANDB_START_METHOD"] = "thread"

강사가 미리 만들어놓은 코드가 들어있는 파일들

In [ ]:
from data_utils import dataset_split
from config_utils import flatten_dict, register_config, configure_optimizers_from_cfg, get_loggers, get_callbacks

## Base 모델 정의

In [ ]:
# Define Model
class BaseGenerativeModel(pl.LightningModule):
    def __init__(self, cfg: DictConfig):
        pl.LightningModule.__init__(self)
        self.cfg = cfg
    
    @abstractmethod
    def forward(self, x):
        raise NotImplementedError()

    def sample_generate(self, *args, **kwargs):
        raise NotImplementedError()
    
    def loss_function(self, *args, **kwargs):
        pass

    def configure_optimizers(self):
        self._optimizers, self._schedulers = configure_optimizers_from_cfg(
            self.cfg, self
        )
        return self._optimizers, self._schedulers

    def training_step(self, batch, batch_idx):
        pass
    
    def validation_step(self, batch, batch_idx):
        pass

class VanillaVAE(BaseGenerativeModel):
    def __init__(self, cfg: DictConfig):
        super.__init__(cfg)
        self.latent_dim = cfg.model.latent_dim

        # mlp를 이용할 예정
        # mlp_module 부분을 꼭 mlp가 아니라 cnn 등 다른 거 사용하면서 실험해 볼 것

        # define posterior (encoder) modules
        posterior_mlp_modules_list = []
        prev_dim = cfg.model.posterior.hidden_dims[0]

        for h_dim in cfg.model.posterior.hidden_dims[1:]:
            posterior_mlp_modules_list.append(nn.Linear(prev_dim, h_dim))
            prev_dim = h_dim

        # * : argument 하나씩 풀어서 입력
        self.posterior_mlp_modules = nn.Sequential(
            *posterior_mlp_modules_list
        )

        # define latent encode
        self.posterior_mu = nn.Linear(
            cfg.model.posteriror.hidden_dims[-1], cfg.model.latent_dim
        )

        self.posteriorlog_log_var = nn.Linear(
            cfg.model.posterior.hidden_dims[-1], cfg.model.latent_dim
        )

        # define prior (decoder) modules
        prior_mlp_modules_list = []
        prev_dim = self.latent_dim
        for h_dim in cfg.model.prior.hidden_dims:
            prior_mlp_modules_list.append(nn.Linear(prev_dim, h_dim))
            prev_dim = h_dim
        self.prior_mlp_modules = nn.Sequential(*prior_mlp_modules_list)

    def encode(self, input):
        hidden = self.posterior_mlp_modules(input)
        mu = self.posterior_mu(hidden)
        log_var = self.posterior_log_var(hidden)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        epsilon = torch.randn_like(std)
        return mu + (epsilon * std)

    def decode(self, z):
        result = self.prior_mlp_modules(z)
        return torch.sigmoid(result)

    def forward(self, x):
        # 아직 코드 안 짰을 때 일단 이렇게 해 놓기
        # raise NotImplementedError()

        # return reconstruction, mu, log_variance
        # -> 결과적으로 reconstruction, latent variable을 return 해야 함

        # input : [batch, 1, 28, 28]
        input = x.view(-1, self.cfg.model.posterior.hidden_dims[0])
        mu, log_var = self.encode(input)
        z = self.reparameterize(mu, log_var)
        return self.decode(z), mu, log_var

    def sample_generate(
        self,
        num_samples: int = 64,  # for training (*batch_size) or random sampling
        z: Optional[torch.Tensor] = None,  # for manual sample generation
        ):
            # raise NotImplementedError()
            if z is None:
                z = torch.randn(num_samples, self.latent_dim)  # num_samples : batch size
            else:
                num_samples = z.shape[0]
            assert z.shape[-1] == self.latent_dim
            z = z.to(self.device)     # GPU inference하면 어떻게 될 지 모르니 일단 device 태워 놓기
            samples = self.decode(z) 
            return samples.view(num_samples, self.cfg.data.C, self.cfg.data.H, self.cfg.data.W)   # 이미 flatten 되어 있기 때문에 -1 할 필요 없음
    
    def loss_function(
        self, 
        recons, 
        real_img, 
        mu, 
        log_var, 
        kld_weight, 
        mode="train"
        ) -> dict:  # dictionary 형태로 정의 : 여러 개 loss를 iter할 것이기 때문
            assert mode in ["train", "val", "test"]
            
            # reconstruction loss
            recons_loss = F.binary_cross_entropy(
                recons,                  # input  : mlp 모델은 항상 이 부분이 flatten 되어 있는 상태일 것임
                real_img.view(-1, self.cfg.model.prior.hidden_dims[-1]),    # target : 실제 정답 값 -> 따라서 얘도 flatten 해 줘야 함 (view(-1, ))
                reduction="sum"
            )

            # kld loss
            kld_loss = 0.5 * torch.sum(
                mu.pow(2) + log_var.exp() - log_var - 1
                )
            
            # summation
            loss = recons_loss + kld_weight * kld_loss
            loss_result = {
                # loss를 그냥 가져다 쓰면 너무 커서 시스템이 폭발하게 됨
                # 따라서 특히 generative 모델에서 VAE 생성할 때 전체 학습 데이터만큼 나눠주는 게 일반적
                f"{mode}_loss": loss / self.cfg.data[f"num_{mode}_imgs"],
                # reconstruction loss
                f"{mode}_reconstruction_loss": recons_loss / self.cfg.data[f"num_{mode}_imgs"],
                # kld loss
                f"{mode}_kld_loss": kld_loss / self.cfg.data[f"num_{mode}_imgs"],
            }

            return loss_result
    
    def training_step(self, batch, batch_idx):
        real_img, labels = batch
        recons, mu, log_var = self.forward(real_img)
        loss_results = self.loss_function(
            recons,
            real_img,
            mu,
            log_var,
            kld_weight=self.cfg.model.kld_weight,
            mode="train"
        )

        loss_results["loss"] = loss_results["train_loss"]
        self.log_dict(loss_results)
        return loss_results
    
    def validation_step(self, batch, batch_idx):
        real_img, labels = batch
        recons, mu, log_var = self.forward(real_img)
        loss_results = self.loss_function(
            recons,
            real_img,
            mu,
            log_var,
            kld_weight=self.model.kld_weight,
            mode="val"
        )

        # loss_results["loss"] = loss_results["val_loss"]
        self.log_dict(loss_results)

        # random sample_generate
        # 기존에 있는 test data가 아니라 random sample에서 생성하는 것
        sample_gens = self.sample_generate(real_img.shape[0])  # batch size 만큼 넣음

        # image logging은 tensorboard, wandb 따로 해야 함
        # 여기에서는 wandb에 맞게 코딩할 예정
        # logger에 direct로 접근해서 logging할 예정 : logger.experiment
        # - wandb.logger, tensorboard summary writer와 같은 의미
        # - 두 개 이상을 동시에 하고 있으면 list 형태로 가져옴
        # - 0번째를 wandb라고 가정
        self.logger.experiment[0].log({
            "inputs": wandb.Image(real_img),
            # reconstruct는 flatten 된 형태 -> view(-1) 필요
            # -1 다음에는 channel, height, width 입력
            "recons": wandb.Image(recons.view(-1, self.cfg.data.C, self.cfg.data.H, self.cfg.data.W)),
            "sample_gens": wandb.Image(sample_gens), # sample generator
        })

        return loss_results

## Configuration Def

In [ ]:
# data configs
data_fashion_mnist_cfg = {
    "name": "fahion_mnist",
    "data_root": os.path.join(os.getcwd(), "data"), # data 저장할 곳
    "transforms": [
        {
            "name": "ToTensor",
            # callable한 object를 keyword argument로 받기
            "kwargs": {}
        }
    ],
    "W": 28,
    "H": 28,
    "C": 1,  # 흑백
    "n_class": 10,
}
cfg = OmegaConf.create(data_fashion_mnist_cfg)
print(OmegaConf.to_yaml(cfg))

# model configs
model_mnist_vanilla_vae_cfg = {
    "name": "VanillaVAE",
    "latent_dim": 4,
    # encoder
    "posterior": {
        "hidden_dim": [28*28, 512, 256]
    },
    # decoder
    "prior": {
        "hidden_dim": [256, 512, 28*28]
    },
    "kld_weight": 1,
}

# optimizer configs
# Google "torch optimizer" : https://github.com/jettify/pytorch-optimizer
#    RAdam : https://github.com/jettify/pytorch-optimizer/blob/master/torch_optimizer/radam.py
#    -> 여기 def _init_() 부분 참고해서 작성
# config_utils.py 파일 보면 optimizer를 for loop으로 돌림
#   = optimizers를 list 형태로 여러 개 지정 가능
#   fine tuning, 모델 두 개 동시 학습시킬 때 등에 사용
opt_cfg = {
    "optimizers": [
        {"name": "RAdam",
         "kwargs": {
             "lr": 1e-3,
             "betas": (0.9, 0.999),
             "eps": 1e-8,
             "weight_decay": 0,
             }
         }
    ],
    "lr_schedulers": [
        {"name": None,  # lr_scheduler 안 쓰려면 none이라고 지정 (config_utils.py 참고)
         "kwargs": {}
         }
    ]
}

_merged_cfg_presets = {
    "vanilla_vae_fashion_mnist": {
        "data": data_fashion_mnist_cfg,
        "model": model_mnist_vanilla_vae_cfg,
        "opt": opt_cfg,   # optimizer
    },
}
 
### hydra composition ###

# clear hydra instance
# notebook에서 여러 번 실행하면 이상하게 나올 수 있어서 clear 먼저 해줄 것
# notebook 아니면 이렇게 할 필요 없음
hydra.core.global_hydra.GlobalHydra.instance().clear()  

# register preset configs
register_config(_merged_cfg_presets)

# initializing
hydra.initialize(config_path=None)

# compose
cfg = hydra.compose("vanilla_vae_fashion_mnist")  # 위에서 사용한 _merged_cfg_presets 키 가져오기

####

# override some cfg
run_name = f"{datetime.now().isoformat(timespec='seconds')}-{cfg.model.name}-{cfg.data.name}"
print(OmegaConf.to_yaml(cfg))

## Define train configs
project_root_dir = os.path.join(
    drive_project_root, "runs", "generative-dnn-tutorial_fashion_mnist_runs"  # 현재 우리가 작업 중인 곳
)

save_dir = os.path.join(project_root_dir, run_name)
run_root_dir = os.path.join(project_root_dir, run_name)

# train configs
train_cfg = {
    "train_batch_size": 256,
    "val_batch_size": 64,
    "test_batch_size": 64,
    "train_val_split": [0.9, 0.1],
    "run_root_dir": run_root_dir,
    "trainer_kwargs": {
        "accelerator": "dp", # 하나의 gpu로 할 때는 dp로 하지만 multiple gpu인 경우 ddp 등 설정 가능
        "gpus": "0"  # -> 0번 gpu 사용하기
        # "gpus": None, # -> 일단 CPU로 확인해 보겠다 : CPU로 할 때 에러에 대한 설명이 더 친절함 (특히 pytorch)
        "max_epochs": 50,    
        # 1.0 : train epoch가 끝날 때 validation check을 하겠다
        # 0.5 : train epoch가 절반 돌았을 때 validation check 하겠다
        # integer인 경우 : 몇 step마다 돌 지 설정하는 것
        "val_check_interval": 1.0,
        "log_every_n_steps": 100,   # 100번 step마다 한다
        "flush_logs_every_n_steps": 100,
    },    
}


# logger configs
log_cfg = {
    "loggers": {
        "WandbLogger": { 
            "project": "fastcampus_generative_fashion_mnist_tutorials",
            "name": run_name,
            "tags": ["fastcampus_generative_fashion_mnist_tutorials"],
            "save_dir": run_root_dir,
        },
        "TensorBoardLogger": {
            "save_dir": project_root_dir,
            "name": run_name,
            },
    },
    "callbacks": {
        "ModelCheckpoint": {
            "save_top_k": 3,
            "monitor": "val_loss",
            "mode": "min",
            "verbose": True,
            "dirpath": os.path.join(run_root_dir, "weights"),
            "filename": "{epoch}-{val_loss:.3f}"  # VAE는 accuracy가 없음
            },
        "EarlyStopping": {
            "monitor": "val_loss",
            "mode": "min",
            # 실제 generation 문제는 classification보다 훨씬 불안정한 경우가 많아서
            # patience 값을 10 정도로 더 크게 두거나
            # 아예 없애고 regularize를 추가하기도 함
            "patience": 3,
            "verbose": True,
            }
    }
}

# unlock config & set train, log cfg
# 이 코드 없으면 에러 발생 : Key 'train' is not in struct
OmegaConf.set_struct(cfg, False)

cfg.train = train_cfg
cfg.log = log_cfg

# lock config
# OmegaConf.set_struct(cfg, True)
print(OmegaConf.to_yaml(cfg))

## Data & dataloader

In [ ]:
# get transforms from torch.vision
def get_transform(cfg: DictConfig):
    transforms_list = []
    for tfm in cfg.data.transforms:
        if hasattr(transforms, tfm.name):
            # getattr(transforms, tfm.name)
            # torch vision에서 transforms의 이름이 tfm.name과 같으면 transforms에 있는 함수를 가져오라는 뜻
            transforms_list.append(
                getattr(transforms, tfm.name)(**tfm.kwargs)
            )
        else:
            raise ValueError(
                f"Not supported transform {tfm} in torch.vision.transform"
            )

    return transforms.Compose(transforms_list)

transform = get_transform(cfg)

def get_datasets(cfg: DictConfig, download: bool = True) -> Dict[str, torch.utils.data.Dataset]:
    data_root = cfg.data.data_root
    datasets = {}

    if cfg.data.name == "fashion_mnist":
        fashion_mnist_dataset = FashionMNIST(data_root, download=download, train=True, transform=transform)
        datasets = dataset_split(fashion_mnist_dataset, split=cfg.train.train_val_split)  # dictionary  형태
        datasets["test"] = FashionMNIST(data_root, download=download, train=False, transform=transforms.ToTensor())    
    else:
        raise NotImplementedError("Not supported dataset yet")
    return datasets

datasets = get_datasets(cfg, download=True)  # 얼마 안 크니 download는 True로 함

train_dataset = datasets["train"]
val_dataset = datasets["val"]
test_dataset = datasets["test"]

# save dataset size
cfg.data.num_train_imgs = len(datasets["train"])
cfg.data.num_val_imgs = len(datasets["val"])
cfg.data.num_test_imgs = len(datasets["test"])

# define dataloader
# batch 단위로 데이터를 묶을 예정
train_batch_size = cfg.train.train_batch_size
val_batch_size = cfg.train.val_batch_size
test_batch_size = cfg.train.test_batch_size

# num_workers : 병렬 processing 
# 1로 설정하는 경우 노트북 환경에서 dp, ddp의 pytorch가 에러나는 경우가 있어 일단 0으로 설정함
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=0
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_batch_size, shuffle=False, num_workers=0
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=val_batch_size, shuffle=False, num_workers=0
)

## 모델 선언 및 손실 함수, 최적화(optimizer) 정의, Tensorboard Logger 정의

- 아래 셀 값들 조절해서 다른 모델 만들어 볼 수 있음

### define model

In [ ]:
# model define

def get_pl_model(cfg: DictConfig, checkpoint_path: Optional[str] = None):

    if cfg.model.name == 'VanillaVAE':
        model = VanillaVAE(cfg)
    else:
        raise NotImplementedError()
    
    if checkpoint_path is not None:
        model = model.load_from_checkpoint(cfg=cfg, checkpoint_path=checkpoint_path)
    return model

model = get_pl_model(cfg)
# print(model)

### GPU setting

In [ ]:
# # gpu = None  # 코드 에러 날 때 cpu는 잘 작동하는지 확인하기 위해 gpu=None으로 설정
# gpu = 0   # gpu를 0번 쓰겠다는 의미

In [ ]:
logger = get_loggers(cfg)
callbacks = get_callbacks(cfg)

trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    default_root_dir=cfg.train.run_root_dir,
    num_sanity_val_steps=2,
    **cfg.train.trainer_kwargs
)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/\#fastcampus/runs/dnn-tutorial-fashion-mnist-run

# pytorch lightning으로 학습
trainer.fit(model, train_dataloader, val_dataloader)

## Test

In [ ]:
ckpt_path = os.path.join(
    cfg.log.callbacks.ModelCheckpoint.dirpath,
    "epoch-=8-val_loss=2.999.ckpt"  # 파일 이름 그대로 가져오기
)

model = get_pl_model(cfg, ckpt_path).eval()
print(model)

In [ ]:
def create_interpolation_images(
        model,
        axis1=0,
        axis2=1,
        latent_dim=4,
        save_img_path=None,
        range1=np.arange(-2, 2, 0.2),
        range2=np.arange(-2, 2, 0.2),
    ):
        assert len(range1) == len(range2)
        z = []
        for i in range1:
            for j in range2:
                cur = [0. for _ in range(latent_dim)]
                cur[axis1] = i
                cur[axis2] = j
                z.append(cur)
        
        z = torch.Tensor(z)
        out = model.sample_generate(z=z)
        out = vutils.make_grid(out, nrow=len(range1))

        if save_img_path is None:
            save_img_path = f"interpolation_results_{axis1}vs{axis2}.png"
        
        vutils.save_image(out, save_img_path)

# 모든 경우의 수에 대해 생성
create_interpolation_images(model, 0, 1)
create_interpolation_images(model, 0, 2)
create_interpolation_images(model, 0, 3)
create_interpolation_images(model, 1, 2)
create_interpolation_images(model, 1, 3)
create_interpolation_images(model, 2, 3)